# Postprocessing EURADIM downscaling models

### Background
In scope of the Destiantion Earth Use Case DE370c, two WGANs for downscaling $NO_x$ and $O_3$ have been trained. 
For this, archived 5 km EURAD-IM forecasts between 2012 and 2018 have been used. Although the performance of the downscaling models is unsatisfactory due to a lack of informative predictor information in the archived forecast data, the following Jupyter Notebook allows for a statistical evaluation of the results. 
The deduced results have been used in the deliverables
- D370c.5.5.1 
- D370c.7.2.1

We start by importing the necessary Python-packages:

In [ ]:
import os, sys, glob
base_dir = os.path.join(os.getcwd(), "..")
dirs2path = ["handle_data", "postprocess", "utils"]
for dir2path in dirs2path:
    sys.path.append(os.path.join(base_dir, dir2path))
import json as js
from datetime import datetime as dt
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib as mpl
import cartopy.crs as ccrs
from handle_data_class import prepare_dataset
from all_normalizations import ZScore
from statistical_evaluation import Scores
from postprocess import run_evaluation_time, run_evaluation_spatial
from plotting import plot_downscaling_examples
from other_utils import convert_to_xarray, get_lcc_coords

### Base directories for test dataset and model

The data and model directory are determined with a couple of parameters as follows:
 - `task`: Two options are avialble: 'O3' and 'NOx' 
 - `data_dir`: directory where the test dataset is stored
 - `model_base_dir`: top-level directory where trained downscaling models are saved
 - `output_base_dir`: output directory for results/plots

In [ ]:
model_type = "sha_wgan"

# custom parameters
task = "O3"                           #  "NOx"
data_dir = f"/p/scratch/deepacf/maelstrom/maelstrom_data/ap5/downscaling_destine_aq/euradim_downscaling/{task.lower()}"
model_base_dir = os.path.join(base_dir, "trained_models", "destine_final")
output_base_dir = os.path.join(base_dir, "results")
eps = 0.01                                                       # epsilon parametre for log-transformed data

# derived parameters
expname = f"{model_type}_{task.lower()}"          
downscaling_dataset = f"euradim_{task.lower()}"

model_base = os.path.join(model_base_dir, expname)
norm_dir, model_dir = model_base, os.path.join(model_base, f"{expname}_generator")
plt_dir = os.path.join(output_base_dir, expname)

### Load configuration and trained model

Read configuration files:

In [ ]:
# create output directory
os.makedirs(plt_dir, exist_ok=True)

# read configuration files
md_config_pattern, ds_config_pattern = f"config_{model_type}.json", f"config_ds_{downscaling_dataset}.json"
md_config_file, ds_config_file = glob.glob(os.path.join(model_base, md_config_pattern)), \
                                 glob.glob(os.path.join(model_base, ds_config_pattern))
if not ds_config_file:
    raise FileNotFoundError(f"Could not find expected configuration file for dataset '{ds_config_pattern}' " +
                            f"under '{model_base}'")
else:
    with open(ds_config_file[0]) as dsf:
        print(f"Read dataset configuration file '{ds_config_file[0]}'.")
        ds_dict = js.load(dsf)

if not md_config_file:
    raise FileNotFoundError(f"Could not find expected configuration file for model '{md_config_pattern}' " +
                            f"under '{model_base}'")
else:
    with open(md_config_file[0]) as mdf:
        print(f"Read model configuration file '{md_config_file[0]}'.")
        hparams_dict = js.load(mdf)

named_targets = hparams_dict.get("named_targets", False)

Next, load the trained model:

In [ ]:
# get trained model
trained_model = keras.models.load_model(model_dir, compile=False)

## Prepare dataset 
Prepare the dataset for inference, get the ground truth data and figure out how to process the output (log-transformation, residual approach)

In [ ]:
# get normalization parameters
js_norm = os.path.join(norm_dir, "norm.json")
print("Read normalization file for subsequent data transformation.")
data_norm = ZScore(ds_dict["norm_dims"])
data_norm.read_norm_from_file(js_norm)

# get dataset pipeline for inference
tfds_opts = {"batch_size": ds_dict["batch_size"], "predictands": ds_dict["predictands"], "predictors": ds_dict["predictors"],
             "lshuffle": False, "var_tar2in": ds_dict.get("var_tar2in", None), "named_targets": named_targets, "lrepeat": False, "drop_remainder": False}

tfds_test, test_info = prepare_dataset(data_dir, downscaling_dataset, ds_dict, hparams_dict, "val", tfds_opts["predictands"], 
                                       norm_obj=data_norm, norm_dims=None, shuffle=tfds_opts["lshuffle"], lrepeat=tfds_opts["lrepeat"],
                                       drop_remainder=tfds_opts["drop_remainder"]) 

In [ ]:
# get ground truth data
fval = test_info.get("file")
tar_varname = test_info["varnames_tar"][0]

ds_test = xr.open_dataset(fval)
ground_truth = ds_test[tar_varname.replace("ln", "").replace("_res", "")].astype("float32", copy=True)

# get predictors and predictands
predictors = ds_dict.get("predictors", None)
if predictors is None:
    predictors = [var for var in list(ds_test.data_vars) if var.endswith("_in")]

tfds_opts["predictors"] = predictors
tfds_opts["predictands"] = test_info["varnames_tar"]

# retrieve parameters to handle inference data
varname_base = tar_varname.split("_")[0]
varname_in = f"{varname_base}_in"

if "_res" in tar_varname:
    lresidual_app = True
    
    y_corr = ds_test[varname_in]
else: 
    lresidual_app = False
    
llog = False
if f"ln" in tar_varname: llog = True

print(f"Target data: {tar_varname} \nInput data: {varname_in} \nGround truth data: {tar_varname.replace('ln', '').replace('_res', '')} is ground truth.")
print(f"Residual approach? {lresidual_app}")
print(f"Log transformation? {llog}")

### Inference

In [ ]:
# start inference
print("Start inference on trained model...")
#with tf.device('/CPU:0'):
y_pred = trained_model.predict(tfds_test, verbose=2)

## Postprocess

Turn raw inference data into xr.DataArray and un-do log-transformation and residual approach if required

In [ ]:
y_pred = convert_to_xarray(y_pred, data_norm, tar_varname, ground_truth.coords, ground_truth.dims)
if lresidual_app: y_pred = y_pred + y_corr
if llog: y_pred = eps*np.exp(y_pred)-eps

Write downscaling results to netCDF

In [ ]:
ncfile_out = os.path.join(plt_dir, f"downscaled_{expname}.nc")

print(f"Write inference data to netCDF-file '{ncfile_out}'")
ground_truth.name, y_pred.name = f"{tar_varname}_ref", f"{tar_varname}_fcst"
ds = xr.Dataset(xr.Dataset.merge(y_pred.to_dataset(), ground_truth.to_dataset()))
ds.to_netcdf(ncfile_out)

### Statistical evaluation

The statistical evaluation is performed in terms of the following metrics:
- RMSE
- Bias

These metrics are evaluated by averaging over all dimensions of the test dataset (time, y, x) and by averaging over the time.
The former provides insight into the overall performance of the downscaling model, whereas the latter allows for an assessment on the spatial dependency of the performance (spatial evaluation). <br><br>
We start with the overall evaluation:

In [ ]:
# start evaluation
print(f"Output data on test dataset successfully processed. Start overall evaluation...")

# instantiate score engine for time evaluation (i.e. hourly time series of evalutaion metrics)
score_engine = Scores(y_pred, ground_truth, ds_dict["norm_dims"][1:])

# run evaluation
rmse_all = run_evaluation_time(score_engine, "rmse", "ppbv", plt_dir, value_range=(0., 10.), model_type=model_type, varname=task)
bias_all = run_evaluation_time(score_engine, "bias", "ppbv", plt_dir, value_range=(-1., 1.), ref_line=0.,
                               model_type=model_type, varname=task)

print(f"RMSE on test year: {rmse_all.mean().values: .2f} (+-{rmse_all.std().values: .2f}) ppbV")
print(f"Bias on test year: {bias_all.mean().values: .2f} (+-{bias_all.std().values: .2f}) ppbV")

Start spatial evaluation:

In [ ]:
# PROJ-parameters corresponding to the 1x1 km**2 target domain
proj_params = {'central_latitude': 54.,
               'central_longitude': 12.5,
               'standard_parallels': (30., 60),
               "false_easting": 450499.7817318188,
               "false_northing": 357999.42235067615}

# Create a Lambert Conformal Conic projection object
proj = ccrs.LambertConformal(**proj_params)
proj_params["earth_radius"] = 6370000.

# Do spatial evaluation
score_engine = Scores(y_pred, ground_truth, [])

lvl_rmse = np.array([0., 0.25, .5, 1.] + list(np.arange(2., 11.1, 1)))
cmap_rmse = mpl.cm.afmhot_r(np.linspace(0., 1., len(lvl_rmse)))
_ = run_evaluation_spatial(score_engine, "rmse", os.path.join(plt_dir, "rmse_spatial"), 
                           dims=ds_dict["norm_dims"][1::], cmap=cmap_rmse, levels=lvl_rmse,
                           projection=proj, extent=[5.5, 8., 50.5, 52.])

Comparison against the bilinearly interpolated input data serving as simple baseline:

In [ ]:
# baseline bilinear interpolation
score_engine = Scores(ds_test[varname_in.replace("ln", "").replace("_res", "")], ground_truth, ds_dict["norm_dims"][1:])

rmse = score_engine("rmse")
bias = score_engine("bias") 

print(f"RMSE on test year: {rmse.mean().values: .2f} (+-{rmse.std().values: .2f}) ppbV")
print(f"Bias on test year: {bias.mean().values: .2f} (+-{bias.std().values: .2f}) ppbV")

### Plot examples 

With the following, specific examples from the test dataset can be plotted.

In [ ]:
# auxiliary variables
lon, lat = get_lcc_coords(ds_test["x"], ds_test["y"], proj_params)

Different time step indices can be selected. The rest will be handled automatically.

In [ ]:
%matplotlib inline

# custom parameter
tstep = 4800 
levels = np.arange(0., 40.)

# get target filename and datetime of sample
tnow = pd.to_datetime(y_pred.isel({'time': tstep})['time'].values)
fname = os.path.join(plt_dir, f"downscaling_{model_type}_{tnow.strftime('%Y%m%d_%H00')}.png")

plot_downscaling_examples(ds_test[varname_in.replace("ln", "")].isel({"time": tstep}).values, ground_truth.isel({"time": tstep}).values,
                         y_pred.isel({"time": tstep}).values, lon, lat, fname, varname="NOx", levels=levels, proj_data=proj)